In [0]:
from functools import reduce
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [0]:
dim_localidade = spark.table("gov_br_combustivel.silver_layer.dim_localidade")
dim_produto = spark.table("gov_br_combustivel.silver_layer.dim_produto")
dim_revenda = spark.table("gov_br_combustivel.silver_layer.dim_revenda")
dim_tempo = spark.table("gov_br_combustivel.silver_layer.dim_tempo")
fact_fuel = spark.table("gov_br_combustivel.silver_layer.fact_fuel")

In [0]:
view_preco_produto = (
    fact_fuel.groupBy("Produto")
    .agg(
        F.avg("ValorVenda").alias("media_valor_venda"),
        F.avg("ValorCompra").alias("media_valor_compra"),
        F.min("ValorVenda").alias("min_valor_venda"),
        F.max("ValorVenda").alias("max_valor_venda"),
        F.min("ValorCompra").alias("min_valor_compra"),
        F.max("ValorCompra").alias("max_valor_compra"),
    )
    .withColumn("media_valor_venda", F.round(F.col("media_valor_venda"), 2))
    .withColumn("media_valor_compra", F.round(F.col("media_valor_compra"), 2))
    .withColumn("diferenca_venda_compra", F.round(F.col("media_valor_venda") - F.col("media_valor_compra"), 2))
    .withColumn(
        "margem_lucro_percentual",
        F.round(
            (F.col("media_valor_venda") - F.col("media_valor_compra")) / F.col("media_valor_compra") * 100, 2
        )
    )
    .select(
        "Produto",
        "min_valor_venda",
        "max_valor_venda",
        "min_valor_compra",
        "max_valor_compra",
        "media_valor_venda",
        "media_valor_compra",
        "diferenca_venda_compra",
        "margem_lucro_percentual"
    )
)

In [0]:
display(view_preco_produto)

In [0]:
view_preco_produto.write.format("delta") \
    .mode("overwrite") \
    .option("zorderBy", "Produto") \
    .saveAsTable("gov_br_combustivel.gold_layer.view_preco_produto")